# Fe test

In [1]:
import sys
sys.path.append('..')

In [2]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [3]:
import os
import logging
from pathlib import Path

import numpy as np
import matplotlib.pyplot as plt

from ase import Atoms
from ase.io import read
from asap3 import FullNeighborList

from m_ff.configurations import carve_confs

from m_ff.kernels import TwoBodySingleSpeciesKernel, ThreeBodySingleSpeciesKernel
from m_ff.gp import TwoBodySingleSpeciesGP, ThreeBodySingleSpeciesGP
from m_ff.grid import SingleSpeciesGrid

from m_ff.interpolation import Spline3D, Spline1D
from m_ff.calculators import TwoBodySingleSpecies, ThreeBodySingleSpecies, CombinedSingleSpecies

from m_ff.models import TwoBodySingleSpeciesModel, ThreeBodySingleSpeciesModel, CombinedSingleSpeciesModel

logging.basicConfig(level=logging.INFO)

## Parameters

In [4]:
# Parameters
directory = Path('data/Fe_vac/')
r_cut = 4.45
sigma_2b = 0.3
sigma_3b = 0.6
noise = 0.0001

# GP Parameters
ntr = 50
ntest = 5
combine_2b_3b = True

# mapping
grid_start = 1.5
elementslist = [26]
num_2b = 100
num_3b = 10


## Configurations

In [5]:
# # # ----------------------------------------
# # # Construct a configuration database
# # # ----------------------------------------


# n_data = 50

# filename = directory / 'movie.xyz'

# traj = read(filename, index=slice(None), format='extxyz')

# elements, confs, forces, energies = carve_confs(
#     traj, r_cut, n_data,
#     forces_label='force', energy_label='energy')

# if not os.path.exists(directory):
#     os.makedirs(directory)

# np.save('{}/confs_cut={:.2f}.npy'.format(directory, r_cut), confs)
# np.save('{}/forces_cut={:.2f}.npy'.format(directory, r_cut), forces)
# np.save('{}/energies_cut={:.2f}.npy'.format(directory, r_cut), energies)

# lens = [len(conf) for conf in confs]

# logging.info('\n'.join((
#     'Number of atoms in a configuration:',
#     '   maximum: {}'.format(np.max(lens)),
#     '   minimum: {}'.format(np.min(lens)),
#     '   average: {:.4}'.format(np.mean(lens))
# )))

## GP

In [6]:
# ----------------------------------------
# Test a simple GP on the built database
# ----------------------------------------


# Get configurations and forces from file
confs = np.load(directory / 'confs_cut={:.2f}.npy'.format(r_cut))
forces = np.load(directory / 'forces_cut={:.2f}.npy'.format(r_cut))
numconfs = len(forces)
ind = np.arange(numconfs)
ind_tot = np.random.choice(ind, size=ntr + ntest, replace=False)

# Separate into random testing and training dataset
# tr_confs, tr_forces = confs[ind[:ntr]], forces[ind[:ntr]]
# tst_confs, tst_forces = confs[ind[ntr:]], forces[ind[ntr:]]

# Use fixed training and testing dataset
tr_confs, tr_forces = confs[:ntr], forces[:ntr]
tst_confs, tst_forces = confs[-ntest - 1:-1], forces[-ntest - 1:-1]

In [7]:
model = CombinedSingleSpeciesModel(elementslist[0], r_cut, sigma_2b, sigma_3b, r_cut / 10.0, r_cut / 10.0, noise)
model.fit(tr_confs, tr_forces)

INFO:m_ff.kernels.twobody:Started compilation of theano two body single species kernels
INFO:m_ff.kernels.twobody:Ended compilation of theano two body single species kernels
INFO:m_ff.kernels.threebodykernel:Started compilation of theano three body kernels
INFO:m_ff.kernels.threebodykernel:Ended compilation of theano three body kernels


KeyboardInterrupt: 

In [ ]:
# Test the GP performance
print('Testing GP')

gp_forces = np.zeros((ntest, 3))
gp_error = np.zeros((ntest, 3))

for i in np.arange(ntest):
    gp_forces[i, :] = model.predict(np.reshape(tst_confs[i], (1, len(tst_confs[i]), 5)))
    gp_error[i, :] = gp_forces[i, :] - tst_forces[i, :]

MAEF = np.mean(np.sqrt(np.sum(np.square(gp_error), axis=1)))
SMAEF = np.std(np.sqrt(np.sum(np.square(gp_error), axis=1)))

MF = np.mean(np.linalg.norm(tst_forces, axis=1))

print('MAEF on forces: {:.4f} +- {:.4f}'.format(MAEF, SMAEF))
print('Relative MAEF on forces: {:.4f} +- {:.4f}'.format(MAEF / MF, SMAEF / MF))

## Mapping

In [ ]:

grid_2b, grid_3b = model.build_grid(grid_start, num_2b, 30)


In [ ]:
dists_2b = np.linspace(grid_start, r_cut, num_2b)
plt.plot(dists_2b, grid_2b(dists_2b, nu=0))
plt.plot(dists_2b, grid_2b(dists_2b, nu=1))

In [ ]:
dists_3b = np.linspace(grid_start, r_cut, 100)

z_min, z_max = -0.2, 0.2
for i in range(0, num_3b, 2):
    plt.title(dists_3b[i])
    
    dists_k = np.ones_like(dists_3b) * dists_3b[i]
    X, Y = np.meshgrid(dists_3b, dists_3b)

    g_data = grid_3b(X.flatten(), Y.flatten(), np.ones_like(X.flatten()) * dists_3b[i]).reshape(100, 100)

    plt.pcolor(dists_3b, dists_3b, -g_data, cmap='RdBu', vmin=z_min, vmax=z_max)
#     plt.pcolor(dists_3b, dists_3b, -g_data[:,:,i], cmap='RdBu', vmin=z_min, vmax=z_max)
    plt.colorbar()
    plt.axis('equal')

    plt.show()

## ASE Calculator

In [ ]:

def rmse(x1, x2):
    rms = np.sqrt(np.sum(np.square(x1 - x2), axis=1))
    return np.mean(rms), np.std(rms)
    

In [ ]:
# grid_2b = TwoBodySingleSpeciesGrid.from_file(str(directory / 'MFF_2b_ntr_20_sig_0.30_cut_4.45.npy'))
# grid_3b = ThreeBodySingleSpeciesGrid.from_file(str(directory / 'MFF_3b_ntr_20_sig_0.60_cut_4.45.npy'))

# calc = ThreeBodySingleSpecies(r_cut=4.45, grid_2b=grid_2b, grid_3b=grid_3b)


# filename = directory / 'movie.xyz'
# traj = read(str(filename), index=slice(0,5))
# for atoms in traj[0:5]:

#     atoms.set_calculator(calc)

#     rms_mean, rms_std = rmse(atoms.arrays['force'], atoms.get_forces())
#     print('MAEF on forces: {:.4f} +- {:.4f}'.format(rms_mean, rms_std))



In [ ]:
calc = TwoBodySingleSpecies(r_cut, grid_2b)

filename = directory / 'movie.xyz'
traj = read(str(filename), index=slice(0, 5))

for atoms in traj:

    atoms.set_calculator(calc)

    rms_mean, rms_std = rmse(atoms.arrays['force'], atoms.get_forces())
    print('MAEF on forces: {:.4f} +- {:.4f}'.format(rms_mean, rms_std))
    
f2 = atoms.get_forces()

In [ ]:
calc = CombinedSingleSpecies(r_cut, grid_2b, grid_3b)

filename = directory / 'movie.xyz'
traj = read(str(filename), index=slice(0, 5))

for atoms in traj:

    atoms.set_calculator(calc)

    rms_mean, rms_std = rmse(atoms.arrays['force'], atoms.get_forces())
    print('MAEF on forces: {:.4f} +- {:.4f}'.format(rms_mean, rms_std))

f23 = atoms.get_forces()